In [1]:
import pandas as pd

data_path= 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\342.html'

df = pd.read_html(data_path)
begin_test_condition = df[0]['Details'].iloc[0]
print(begin_test_condition)
#display(df)
# display(df)
# print(type(df))

BT_AustraliaNewSubmission


In [2]:
#display(df[0]['Status'].iloc[-1])# print(len(df))
print(type(df[2]))
display(df[30])



# print(df[0].Status.value_counts()30
'cancel' in df[30]['Status'].tolist()

<class 'pandas.core.frame.DataFrame'>


,Status,Timestamp,Details
0,info_outline,12:41:09 PM,Clicked on 'PricingSummaryPage_bindOptions_But...
1,info_outline,12:41:10 PM,Clicked on 'PricingSummaryPage_bindSubmission_...
2,info_outline,12:41:10 PM,Alert is not present to accept
3,info_outline,12:41:14 PM,Web Element is present 'PricingSummaryPage_bin...
4,check_circle,12:41:14 PM,After clicking on 'Bind' - Checking Web Elemen...
5,info_outline,12:41:14 PM,Clicked on 'PricingSummaryPage_bind_Button'
6,info_outline,12:41:32 PM,Web Element is not present 'PricingSummaryPage...
7,info_outline,12:41:37 PM,Web Element is not present 'PricingSummaryPage...
8,cancel,12:42:38 PM,After Bind Submission - Checking Web Element '...
9,cancel,12:42:38 PM,java.lang.Exception: After Bind Submission - C...


True

In [3]:
'''
Workflow
1- Sumar pasos
2 - Checkear si el ultimo es un cancel
   2.1 - Si es un cancel, guardar la informacion en la primera fila del DF_N
   2.2 - Si no es un cancel, checkear si el test acabo
       2.2.1 - Si aun no acabo sumar los pasos a los que habia antes
       2.2.2 - Guardar los datos en el DF y comenzar una fila nueva
       
       
TODO

Hacer cambios para poder trabajar con varios workflows

'''

# On this list we will be adding our dictionaries to improve speed
rows_list = []
steps = 0
start_time=0
end_time=0
print(df[0]['Details'].iloc[0])

# This dictionary will be the one filling the rows_list
dict_test = {}
df_extracted = pd.DataFrame(columns=['Steps','Status','Start Time', 'End Time'])
# Boolean to check if we are still on a test case
in_test = False
 


for i in range(len(df)-1):
    #Starting a test case
    
    # This if statement is the weakest part, should modify for multi work-flows
    if len(df[i])==2 and df[i]['Timestamp'].iloc[0]==df[i]['Timestamp'].iloc[-1]:
        #import pdb; pdb.set_trace()
        pass
    else:
        # This if statement is the weakest part, should modify for multi work-flows
        if "Loaded the URL" in df[i]['Details'].iloc[0] and df[i]['Status'].sum()!=1:
            print('first time it enters in a test')
          #  import pdb;pdb.set_trace()
            # If we have a fail on the first mayor step, it will stop here and save the test in the list
            if 'cancel' in df[i]['Status'].tolist():
                # Values of the test
                steps = df[i].Status.value_counts()['info_outline']
                start_time = df[i]['Timestamp'].iloc[0]
                end_time   = df[i]['Timestamp'].iloc[-1]
                status = 'fail'

                # Add a new row
                dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                rows_list.append(dict_test)
                dict_test = {}
                in_test = False
                
            # If we are on one work-flow this will make sure to save the previous test case
            # before getting on the next one
            if in_test:
                #import pdb;pdb.set_trace()
                status = 'pass'
                #import pdb; pdb.set_trace()
                # Add a new row
                dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                rows_list.append(dict_test)
                dict_test = {}
                in_test = False

            
            # Recopilar pasos

            # The first time it enters a test
            else:
                start_time = df[i]['Timestamp'].iloc[0]
                end_time = df[i]['Timestamp'].iloc[-1]
                steps = df[i].Status.value_counts()['info_outline']
                in_test = True

                # If it is the same 
                if i==(len(df)-2):
                    status = 'pass'
                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}

                    
        #We will enter here if there is continuation on a text or if the test was fatal
        else:
            if df[i].Status.value_counts().sum()==1 and ('redo' in df[i]['Status'].tolist() or 'cancel' in df[i]['Status'].tolist()):
               # import pdb;pdb.set_trace()
                if 'cancel' in df[i].Status.value_counts().tolist():
                    status = 'fatal'
                else:
                    status = 'skip'
                steps = 1
                start_time = df[i]['Timestamp'].iloc[0]
                end_time   = df[i]['Timestamp'].iloc[0]

                # Add a new row
                dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                rows_list.append(dict_test)
                dict_test = {}
            else:
                # Continuing the test case but it is a failed test
                if 'cancel' in df[i]['Status'].tolist():
                    #print('seems to be working')
                  #  import pdb;pdb.set_trace()
                    steps += df[i].Status.value_counts()['info_outline']
                    end_time = df[i]['Timestamp'].iloc[-1]
                    status = 'fail'

                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                    in_test = False

                # Continuing the test case
                else:
                   # print('continue on the test testing')
                    #import pdb;pdb.set_trace()
                    try:
                        steps += df[i].Status.value_counts()['info_outline']
                        end_time = df[i]['Timestamp'].iloc[-1]
                    except:
                        import pdb;pdb.set_trace()
                    in_test = True
                    if i==(len(df)-2):
                       # import pdb;pdb.set_trace()
                        status = 'pass'
                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}

                    # Añadir al dataframe

df_middle = pd.DataFrame(rows_list)
# df_extraced = pd.concat([df_extracted, df_middle])

display(df_middle)

BT_AustraliaNewSubmission
first time it enters in a test
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
continue on the test testing
seems to be working
first time it enters in a test
continue on the test testing
continue on the test testing
continue on the test t

,Steps,Status,Start Time,End Time
0,226,fail,12:35:20 PM,12:42:38 PM
1,205,fail,12:42:40 PM,12:48:41 PM
2,139,pass,12:48:43 PM,12:52:28 PM
3,267,fail,12:48:43 PM,12:59:05 PM
4,139,fail,12:59:07 PM,1:04:46 PM
5,202,fail,1:04:48 PM,1:11:13 PM
6,141,fail,1:11:16 PM,1:15:52 PM
7,1,fatal,1:15:53 PM,1:15:53 PM


In [43]:
import os

# Read all the file names of a given directory
data_path = 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\' # Cambiar esto
dirs = os.listdir(data_path)

# Open all the excel files and concatenate the data into a single variable
df = pd.DataFrame()
for name in dirs:
    print(data_path+name)
    df_name = pd.read_html(data_path+name)
    extracted = add_report(df_name)
    df = pd.concat([df, extracted], axis=0)

C:\Users\c.hernandezperez\Desktop\drop\reports\index.html
Loaded the URL : 'https://owl-gatewaywebapp-uat.azurewebsites.net/contratar-seguro-coche'
> <ipython-input-25-9d5cee48c046>(41)add_report()
-> dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
(Pdb) 
(Pdb) exit()


BdbQuit: 